# Trabajo práctico número 1

## Hacer un programa que utilice un Algoritmo Genético Canónico para buscar un máximo de la función:

#### f(x) = (x/coef)2 en el dominio [0 , 2^(30 - 1)]
donde coef = 2^(30-1)
teniendo en cuenta los siguientes datos:

1– Probabilidad de Crossover = 0,75

2– Probabilidad de Mutación = 0,05

3– Población Inicial: 10 individuos

4– Ciclos del programa: 200

5– Método de Selección: Ruleta

6– Método de Crossover: 1 Punto

7– Método de Mutación: invertida

#### El espacio solución estara formado por cromosomas de longitud 30
 y cada gen sera un dígito
binario 0 o 1

### Constantes

In [384]:
PROB_CROSSOVER = 0.75
PROB_MUTACION = 0.05
CICLOS = 200
CANTIDAD_POBLACION = 10
COEF = 1073741823 # 2^29

### Librerías necesarias

In [385]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Generando población inicial aleatoria

In [386]:
rng = np.random.default_rng(12345) # Random Number Generator
# El 12345 es la semilla, poner sin semilla para q en cada ejecución genere nuevos aleatorios.
poblacion_decimal = rng.integers(low=0, high=COEF, size=CANTIDAD_POBLACION)
poblacion_decimal

array([ 750776422,  244100195,  846803254,  340116676,  219233270,
        856164639,  690076227,  726122922, 1061347010,  419950681],
      dtype=int64)

#### Convirtiendo la población inicial a binario

In [401]:
def convertirPoblacionBin(pob_dec):
    pob_bin = []
    contador = 0
    for decimal in pob_dec:
        # convirtiendo cada número decimal en binario de 8 dígitos.
        pob_bin.append(format(decimal,'030b'))
    return pob_bin

poblacion_binarios= convertirPoblacionBin(poblacion_decimal)
poblacion_binarios

['101100101111111111000001100110',
 '001110100011001010110001100011',
 '110010011110010011000100110110',
 '010100010001011100010011000100',
 '001101000100010011101111110110',
 '110011000010000000100100011111',
 '101001001000011011101001000011',
 '101011010001111100000110101010',
 '111111010000101101111011000010',
 '011001000001111111000001011001']

### Función objetivo

In [389]:
#(x/coef)2
# Pasar población decimal
def funcionObjetivo(pob):
    colObjetivo=[]
    # El nombre cromosoma es por semántica, en realidad va a trabajar con cada valor decimal
    for cromosoma in pob:
        colObjetivo.append((int(cromosoma)/int(COEF))**2)
    #convertir lista en numpy arrray (para operaciones matemáticas)
    npColObjetivo = np.array(colObjetivo)
    return npColObjetivo

### Generando un marco de datos

In [390]:
def generarDataFrame(pob_dec, pob_bin):
    data = {'Decimales': pob_dec, 'Binarios': pob_bin, 'FuncionObjetivo':funcionObjetivo(pob_dec)}
    dataFrame = pd.DataFrame(data)
    dataFrame['Fitness'] = dataFrame['FuncionObjetivo'] / dataFrame['FuncionObjetivo'].sum()
    return dataFrame

marcoDeDatos = generarDataFrame(poblacion_decimal, poblacion_binarios)
display(marcoDeDatos)

,Decimales,Binarios,FuncionObjetivo,Fitness
0,750776422,101100101111111111000001100110,0.488902,0.124063
1,244100195,001110100011001010110001100011,0.051682,0.013115
2,846803254,110010011110010011000100110110,0.621964,0.157829
3,340116676,010100010001011100010011000100,0.100336,0.025461
4,219233270,001101000100010011101111110110,0.041688,0.010579
5,856164639,110011000010000000100100011111,0.635792,0.161338
6,690076227,101001001000011011101001000011,0.413042,0.104813
7,726122922,101011010001111100000110101010,0.457320,0.116049
8,1061347010,111111010000101101111011000010,0.977046,0.247935
9,419950681,011001000001111111000001011001,0.152967,0.038817


#### Estadisticas

In [391]:
def generarEstadisticas(dataFrame):
    stats = {'Suma':[ dataFrame['FuncionObjetivo'].sum(), dataFrame['Fitness'].sum() ], 'Promedio': [ dataFrame['FuncionObjetivo'].mean(), dataFrame['Fitness'].mean() ], 'Maximo': [ dataFrame['FuncionObjetivo'].max(), dataFrame['Fitness'].max() ]}
    dataFrame_stats = pd.DataFrame(stats, index = ['Función objetivo', 'Fitness'])
    return dataFrame_stats

marcoDeDatos_stats = generarEstadisticas(marcoDeDatos)

display(marcoDeDatos_stats)

,Suma,Promedio,Maximo
Función objetivo,3.940738,0.394074,0.977046
Fitness,1.000000,0.100000,0.247935


### Generando una nueva población a partir de otra

Selección de padres para la próxima generación

#### Método ruleta

In [392]:
def ruleta(dataFrame): #pasar marco de datos
    probabilidades = np.array(dataFrame['Fitness']) #Asigna probabilidad basada en el fitness
    cromosomas = np.array(dataFrame['Binarios'])
    np.random.seed() 
    salida = np.random.choice(cromosomas, size=int(CANTIDAD_POBLACION), p=probabilidades)
    return salida

cromosomasPadresRuleta = ruleta(marcoDeDatos)
cromosomasPadresRuleta

array(['110011000010000000100100011111', '101001001000011011101001000011',
       '101100101111111111000001100110', '101001001000011011101001000011',
       '110011000010000000100100011111', '111111010000101101111011000010',
       '111111010000101101111011000010', '111111010000101101111011000010',
       '110010011110010011000100110110', '101100101111111111000001100110'],
      dtype=object)

#### Método torneo

In [393]:
def torneo(dataFrame):
    salida = []
    fitness = np.array(dataFrame['Fitness'])
    cromosomas = np.array(dataFrame['Binarios'])
    for i in range(0,10):
        posiblesCantidades = [ x for x in range(1,11)]
        np.random.seed() 
        cantidadMiembrosTorneo = np.random.choice(posiblesCantidades, size=1)
        miembrosTorneo = np.random.choice(fitness, size=cantidadMiembrosTorneo[0], replace=False) #Trabaja directamente con los fitness de los cromósomas
        ganador = np.amax(miembrosTorneo)
        indiceGanador = np.where(fitness == ganador) # retorna ndarray
        cromosomaGanador = cromosomas[indiceGanador[0][0]]
        salida.append(cromosomaGanador)
    return salida

cromosomasPadresTorneo = torneo(marcoDeDatos)
cromosomasPadresTorneo

['111111010000101101111011000010',
 '101011010001111100000110101010',
 '111111010000101101111011000010',
 '110010011110010011000100110110',
 '111111010000101101111011000010',
 '111111010000101101111011000010',
 '111111010000101101111011000010',
 '111111010000101101111011000010',
 '101100101111111111000001100110',
 '110011000010000000100100011111']

Test de prueba (podemos borrar desp)

In [394]:
""" fitness = np.array(marcoDeDatos['Fitness'])
cromosomas = np.array(marcoDeDatos['Binarios'])
posiblesCantidades = [ x for x in range(1,11)]
cantidadMiembrosTorneo = np.random.choice(posiblesCantidades, size=1)
miembrosTorneo = np.random.choice(fitness, size=cantidadMiembrosTorneo[0], replace=False) #Trabaja directamente con los fitness de los cromósomas
ganador = np.amax(miembrosTorneo)
indiceGanador = np.where(fitness == ganador)
print(miembrosTorneo)
print("Cantidad: "+str(cantidadMiembrosTorneo[0]))
print("Fitness ganador: "+str(ganador))
print("Indice: "+str(indiceGanador[0][0]))
print("Cromosoma: "+str(cromosomas[indiceGanador[0][0]])) """

' fitness = np.array(marcoDeDatos[\'Fitness\'])\ncromosomas = np.array(marcoDeDatos[\'Binarios\'])\nposiblesCantidades = [ x for x in range(1,11)]\ncantidadMiembrosTorneo = np.random.choice(posiblesCantidades, size=1)\nmiembrosTorneo = np.random.choice(fitness, size=cantidadMiembrosTorneo[0], replace=False) #Trabaja directamente con los fitness de los cromósomas\nganador = np.amax(miembrosTorneo)\nindiceGanador = np.where(fitness == ganador)\nprint(miembrosTorneo)\nprint("Cantidad: "+str(cantidadMiembrosTorneo[0]))\nprint("Fitness ganador: "+str(ganador))\nprint("Indice: "+str(indiceGanador[0][0]))\nprint("Cromosoma: "+str(cromosomas[indiceGanador[0][0]])) '

#### Crossover

In [395]:
def crossover(padres): #Pasar ndarray cromosomas padres
    hijos = []
    for i in range(0,10,2):
        opciones=[True, False]
        np.random.seed() 
        prob_cross = np.array([PROB_CROSSOVER,(1-PROB_CROSSOVER)]) # Probabilidades de cada opción
        cross = np.random.choice(opciones, size=1, p=prob_cross)
        if cross[0]:
            posiciones = [x for x in range(0, 30)]
            probCorte = [1/30 for x in range(0, 30)]
            corte = np.random.choice(posiciones, size=1, p=probCorte) # Devuelve ndarray de 1 elemento
            posicionCorte = corte[0] 
            primerTiraGenesPadre1 = padres[i][0:posicionCorte]
            segundaTiraGenesPadre1 = padres[i][posicionCorte:30]
            primerTiraGenesPadre2 = padres[i+1][0:posicionCorte]
            segundaTiraGenesPadre2 = padres[i+1][posicionCorte:30]
            hijos.append(primerTiraGenesPadre1 + segundaTiraGenesPadre2) # Hijo 1
            hijos.append(primerTiraGenesPadre2 + segundaTiraGenesPadre1) # Hijo 2
        else:
            hijos.append(padres[i])
            hijos.append(padres[i+1])
    return hijos

##### Crossover ruleta

In [396]:
cromosomasHijosRuleta = crossover(cromosomasPadresRuleta)
cromosomasHijosRuleta

['110011000010000000100100011111',
 '101001001000011011101001000011',
 '101100101111111111000001100011',
 '101001001000011011101001000110',
 '110011000010000000100100011111',
 '111111010000101101111011000010',
 '111111010000101101111011000010',
 '111111010000101101111011000010',
 '110010011110010011000100110110',
 '101100101111111111000001100110']

##### Crossover torneo

In [399]:
cromosomasHijosTorneo = crossover(cromosomasPadresTorneo)
print(cromosomasHijosTorneo)

['111111010000111100000110101010', '101011010001101101111011000010', '111111010000101101111100110110', '110010011110010011000011000010', '111111010000101101111011000010', '111111010000101101111011000010', '111111010000101101111011000010', '111111010000101101111011000010', '101100101111000000100100011111', '110011000010111111000001100110']


#### Mutación

In [398]:
def mutacion(hijos):
    # TODO metodo
    return hijos 